<a href="https://colab.research.google.com/github/JulianSKYOO/Projects/blob/main/(OCT_2024)%5BLLM%5DNPC_Emotion_Guess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NPC Emotion Guess
created by Sungkyn Yoo

# Overview
With training the model with Large Language Model `FLAN-T5`, we can generate the completion by the model guessing how the NPC would feel throught the dialogue.

# Dataset
The [dataset](https://huggingface.co/datasets/amaydle/npc-dialogue) from the hugging face provided

Name / Name of NPC

Biography / Brief info and a bio of NPC

Query / Query generated by the user

Response / Reseponse generated by the corresponding query

Emotion / Emotion of NPC after receiving the corresponding Response by the user

In [67]:
#@title Checking AWS Instance Kernel

import os

instance_type_req = ' ' # corresponding instance
instance_type_cur = os.environ.get('HOSTNAME')

assert instance_type_req in instance_type_cur, f"Instance type mismatch. Required: {instance_type_req}, Current: {instance_type_cur}"
print(f"Instance correct")

AssertionError: Instance type mismatch. Required:  , Current: fd9400703315

### Install the packages needed, PyTorch and Transformers

In [1]:
!pip install datasets==2.17.0

!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==2.0.0 \
    torchdata==0.6.0 --quiet

!pip install \
    transformers==4.27.2 --quiet \
    rouge_score==0.1.2 \
    evaluate==0.4.0 --quiet \
    loralib==0.1.1 \
    peft==0.3.0 --quiet \
    #accelerate==0.21.0 \
    #bitsandbytes==0.40.2 --quiet

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.10.0 which is incompatible.
  Using cached 

This is the Reinforcement Learning library created by Leandro Von Werra [here](https://github.com/lvwerra)

In [3]:
!pip install git+https://github.com/lvwerra/trl.git@25fa1bd

  Cloning https://github.com/lvwerra/trl.git (to revision 25fa1bd) to /tmp/pip-req-build-h7if9e29
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-h7if9e29
  Running command git checkout -q 25fa1bd
  Resolved https://github.com/lvwerra/trl.git to commit 25fa1bd
  Preparing metadata (setup.py) ... done
  Created wheel for trl: filename=trl-0.4.2.dev0-py3-none-any.whl size=67531 sha256=9f752c57d20520d83cf292f01401596322ff2cf44202506a20d8f6ae3348854c
  Stored in directory: /tmp/pip-ephem-wheel-cache-c6p1wh7g/wheels/24/b4/20/2fa3a1e47c0411c39e198029315e3af2a2c1d59132913f136f
Successfully built trl


### Necessary Imports

In [5]:
from datasets import load_dataset

from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig, TrainingArguments, Trainer
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import time
import evaluate

import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

In [6]:
hugface_dataset_name = "amaydle/npc-dialogue"

dataset = load_dataset(hugface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1723 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/192 [00:00<?, ? examples/s]

In [55]:
dataset.shape

{'train': (1723, 5), 'test': (192, 5)}

In [15]:
dataset.column_names

{'train': ['Name', 'Biography', 'Query', 'Response', 'Emotion'],
 'test': ['Name', 'Biography', 'Query', 'Response', 'Emotion']}

In [18]:
example = [20, 100]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example):
    print(dash_line)
    print("Example ", i + 1)
    print(dash_line)
    print('Name:')
    print(dataset['test'][index]['Name'])
    print(dash_line)
    print('Biography:')
    print(dataset['test'][index]['Biography'])
    print(dash_line)
    print('Query:')
    print(dataset['test'][index]['Query'])
    print(dash_line)
    print('Response:')
    print(dataset['test'][index]['Response'])
    print(dash_line)
    print('Emotion:')
    print(dataset['test'][index]['Emotion'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Name:
Arin
---------------------------------------------------------------------------------------------------
Biography:
Arin is a mischievous and cunning elf who grew up in the magical forest. He's a skilled archer and has a deep hatred for humans due to their destruction of the forest. He joined the player's quest for revenge.
---------------------------------------------------------------------------------------------------
Query:
"What's your favorite color?"
---------------------------------------------------------------------------------------------------
Response:
"I don't have time for frivolous questions."
---------------------------------------------------------------------------------------------------
Emotion:
Fear
---------------------------------------------------

`FLAN-T5` model to create Seq2Seq Language Model
and tokenizer, [link](https://huggingface.co/docs/transformers/v4.45.2/en/model_doc/auto#transformers.AutoTokenizer)

In [19]:
model_name="google/flan-t5-base"

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Testing Tokenizer for a sample sentence.

In [26]:
sample_sentence = "Songbird returned to NUSA."

enc_sentence = tokenizer(sample_sentence, return_tensors='pt')

dec_sentence = tokenizer.decode(enc_sentence['input_ids'][0], skip_special_tokens=True) # Skipping special tokens is a default.

print(enc_sentence['input_ids'][0])
print(dec_sentence)

tensor([11263,  9414,  3666,    12,   445, 17663,     5,     1])
Songbird returned to NUSA.


In [34]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model params: {trainable_model_params}\nall model params: {all_model_params}\n% of trainable model params: {100* trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model params: 247577856
all model params: 247577856
% of trainable model params: 100.00%


## Model without any prompt engineering

In [35]:
for i, index in enumerate(example):
    diag_npc = dataset['test'][index]['Name'] + \
    ": " + dataset['test'][index]['Biography'] + \
    "\n" + dataset['test'][index]['Query'] + \
    "\n" + "Me: " + dataset['test'][index]['Response']
    emotion = dataset['test'][index]['Emotion']

    input = tokenizer(diag_npc, return_tensors='pt')
    output = tokenizer.decode(
        original_model.generate(
            input['input_ids'],
            max_new_tokens=30
        )[0], skip_special_tokens=True
    )

    print(dash_line)
    print("Example ", i + 1)
    print(dash_line)
    print(f"INPUT PROMPT:\n{diag_npc}")
    print(dash_line)
    print(f"BASELINE GUIDED EMOTION:\n{emotion}")
    print(dash_line)
    print(f"MODEL GENERATION - No Prompt Engineering:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Arin: Arin is a mischievous and cunning elf who grew up in the magical forest. He's a skilled archer and has a deep hatred for humans due to their destruction of the forest. He joined the player's quest for revenge.
"What's your favorite color?"
Me: "I don't have time for frivolous questions."
---------------------------------------------------------------------------------------------------
BASELINE GUIDED EMOTION:
Fear
---------------------------------------------------------------------------------------------------
MODEL GENERATION - No Prompt Engineering:
Arin: Arin: I'm a mischievous and cunning elf who grew up in the magical forest. I'

---------------------------------------------------------------------------------------------------
Example  2
------------

## Instruction Prompt

### Zero-shot Inference with Instruction Prompt

In [36]:
for i, index in enumerate(example):
    diag_npc = dataset['test'][index]['Name'] + \
    ": " + dataset['test'][index]['Biography'] + \
    "\n" + dataset['test'][index]['Query'] + \
    "\n" + "Me: " + dataset['test'][index]['Response']
    emotion = dataset['test'][index]['Emotion']

    prompt = f"""
Guess the NPC's emotion.

{diag_npc}

Emotion:
    """

    input = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        original_model.generate(
            input['input_ids'],
            max_new_tokens=50
        )[0], skip_special_tokens=True
    )

    print(dash_line)
    print("Example ", i + 1)
    print(dash_line)
    print(f"INPUT PROMPT:\n{prompt}")
    print(dash_line)
    print(f"BASELINE GUIDED EMOTION:\n{emotion}")
    print(dash_line)
    print(f"MODEL GENERATION - Zero-shot:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Guess the NPC's emotion.

Arin: Arin is a mischievous and cunning elf who grew up in the magical forest. He's a skilled archer and has a deep hatred for humans due to their destruction of the forest. He joined the player's quest for revenge.
"What's your favorite color?"
Me: "I don't have time for frivolous questions."

Emotion:
    
---------------------------------------------------------------------------------------------------
BASELINE GUIDED EMOTION:
Fear
---------------------------------------------------------------------------------------------------
MODEL GENERATION - Zero-shot:
anger

---------------------------------------------------------------------------------------------------
Example  2
------------------------------------------------------------

### Zero-shot Inference with a different Prompt Template

In [39]:
for i, index in enumerate(example):
    diag_npc = dataset['test'][index]['Name'] + \
    ": " + dataset['test'][index]['Biography'] + \
    "\n" + dataset['test'][index]['Query'] + \
    "\n" + "Me: " + dataset['test'][index]['Response']
    emotion = dataset['test'][index]['Emotion']

    prompt = f"""
Dialogue:

{diag_npc}

What kind of feeling does the NPC have?
    """

    input = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        original_model.generate(
            input['input_ids'],
            max_new_tokens=50
        )[0], skip_special_tokens=True
    )

    print(dash_line)
    print("Example ", i + 1)
    print(dash_line)
    print(f"INPUT PROMPT:\n{prompt}")
    print(dash_line)
    print(f"BASELINE GUIDED EMOTION:\n{emotion}")
    print(dash_line)
    print(f"MODEL GENERATION - Zero-shot:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

Arin: Arin is a mischievous and cunning elf who grew up in the magical forest. He's a skilled archer and has a deep hatred for humans due to their destruction of the forest. He joined the player's quest for revenge.
"What's your favorite color?"
Me: "I don't have time for frivolous questions."

What kind of feeling does the NPC have?
    
---------------------------------------------------------------------------------------------------
BASELINE GUIDED EMOTION:
Fear
---------------------------------------------------------------------------------------------------
MODEL GENERATION - Zero-shot:
hatred

---------------------------------------------------------------------------------------------------
Example  2
-------------------------------------------

### One-shot Inference

In [50]:
def few_shot_prompt(example_full, example_index_to_emotion):
    prompt = ''
    for index in example_full:
        diag_npc = dataset['test'][index]['Name'] + \
        ": " + dataset['test'][index]['Biography'] + \
        "\n" + dataset['test'][index]['Query'] + \
        "\n" + "Me: " + dataset['test'][index]['Response']
        emotion = dataset['test'][index]['Emotion']

        prompt += f"""
Dialogue:

{diag_npc}

How would the NPC feel?
{emotion}


"""

        diag_npc = dataset['test'][example_index_to_emotion]['Name'] + \
        ": " + dataset['test'][example_index_to_emotion]['Biography'] + \
        "\n" + dataset['test'][example_index_to_emotion]['Query'] + \
        "\n" + "Me: " + dataset['test'][example_index_to_emotion]['Response']

        prompt += f"""
Dialogue:

{diag_npc}

How would the NPC feel?
"""

    return prompt

In [51]:
example_full = [20]
example_index_to_emotion = 100

one_shot_prompt = few_shot_prompt(example_full, example_index_to_emotion)

print(one_shot_prompt)


Dialogue:

Arin: Arin is a mischievous and cunning elf who grew up in the magical forest. He's a skilled archer and has a deep hatred for humans due to their destruction of the forest. He joined the player's quest for revenge.
"What's your favorite color?"
Me: "I don't have time for frivolous questions."

How would the NPC feel?
Fear



Dialogue:

Neo: Neo is a computer programmer and a hacker who discovers that the reality he lives in is just a simulated reality created by sentient machines. He is the "One," a prophesied figure who is believed to have the power to change the Matrix and free humanity from enslavement.
Why do you fight?
Me: For those who cannot fight for themselves.

How would the NPC feel?



In [52]:
emotion = dataset['test'][example_index_to_emotion]['Emotion']

input = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        input['input_ids'],
        max_new_tokens=50
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f"BASELINE GUIDED EMOTION:\n{emotion}\n")
print(dash_line)
print(f"MODEL GENERATION - One-shot:\n{output}")

---------------------------------------------------------------------------------------------------
BASELINE GUIDED EMOTION:
Determined

---------------------------------------------------------------------------------------------------
MODEL GENERATION - One-shot:
apathetic


### Four-shot Inferece

In [59]:
example_full = [10, 20, 30, 40]
example_index_to_emotion = 60

four_shot_prompt = few_shot_prompt(example_full, example_index_to_emotion)

print(four_shot_prompt)


Dialogue:

Thalia Windrider: Thalia is a half-elf ranger who grew up in the forest with her elven mother and human father. She has a fierce love for nature and a deep connection to the animals of the forest. Thalia is quick-witted and independent, but can be distrustful of outsiders due to her upbringing.
What is your favorite animal in the forest?
Me: That's a tough question. I love all the animals I encounter, but I have a special fondness for the wolves.

How would the NPC feel?
Happiness



Dialogue:

Elsa: Elsa is the Snow Queen of Arendelle, with the power to control ice and snow. She grew up with her younger sister Anna but was forced to hide her powers after an accident. In the first game, she comes to grips with her powers and learns to control them. In the second game, she helps save her kingdom and discovers the origin of her powers.
What is your favorite season?
Me: Winter, of course!

How would the NPC feel?

Dialogue:

Arin: Arin is a mischievous and cunning elf who grew

In [61]:
emotion = dataset['test'][example_index_to_emotion]['Emotion']

input = tokenizer(four_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        input['input_ids'],
        max_new_tokens=50
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f"BASELINE GUIDED EMOTION:\n{emotion}\n")
print(dash_line)
print(f"MODEL GENERATION - Four-shot:\n{output}")

---------------------------------------------------------------------------------------------------
BASELINE GUIDED EMOTION:
Happy

---------------------------------------------------------------------------------------------------
MODEL GENERATION - Four-shot:
Happy


Already from the four-shot inference the model would generate more accurate completion.

**Caveat**

the current model with `FLAN-T5` has 512 tokens. The context > 512 will be ignored.

**Transformers GenerationConfig**
[link](https://huggingface.co/docs/transformers/en/main_classes/text_generation)

`do_sample=True`: turns on the decoding strategy which influence the next token from the probability distribution over the entire vocabulary.
`temperature=[0,1]`: how much creativity you want the model to have.
`top_p=p`: selecting an output based on random-weighted strategy with a cumulative probability <= p
`top_k=k`: selecting the `k`th output based on random-weighted strategy

* greedy decoding: `num_beams=1` and `do_sample = False`
* contrastive search: `penalty_alpha>0` and `top_k>1`
* multinomial sampling: `num_beams=1` and `do_sample=True`
* beam-search decoding: `num_beams>1` and `do_sample=False`
* beam-search multinomial sampling: `num_beams>1` and `do_sample=True`
* diverse beam-search decoding: `num_beams>1` and `num_beam_groups>1`
* constrained beam-searching decoding: `constraints!=None` or `force_words_ids!=None`
* assisted decoing: `assistant_model` or `prompt_loopup_num_tokens` is passed to `.generate()`

In [65]:
generate_config = GenerationConfig(max_new_token=20, do_sample=True, temperature=0.1, top_k=1)

input = tokenizer(four_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        input['input_ids'],
        generation_config=generate_config
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f"BASELINE GUIDED EMTION:\n{emotion}\n")
print(dash_line)
print(f"MODEL GENERATION - Four-shot:\n{output}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


---------------------------------------------------------------------------------------------------
BASELINE GUIDED EMTION:
Happy

---------------------------------------------------------------------------------------------------
MODEL GENERATION - Four-shot:
Happy


### Full Fine-Tuning